In [3]:
import pandas as pd

In [ ]:
def loading(path: str, name: str) -> pd.DataFrame:
    """
    this function loading crypto index and keep only closed values
    args:
        - path: path to file
        - name: name of index
    return:
        - pandas DataFrame with columns Date and index
    """

    renaming_dict = {
        'Date': 'date', 'Close': name, 'Volume': name + '_vol'
    }

    df = pd.read_csv(path, parse_dates=['Date']).drop(columns=['Adj Close'])

    df = df.rename(columns=renaming_dict)
    df = df[renaming_dict.values()]
    
    df[name+'_chg'] = df[name].pct_change().fillna(0).multiply(100).round(2)
    df[name+'_vol_chg'] = df[name+'_vol'].pct_change().fillna(0).multiply(100).round(2)
    for col in df.columns[1:]:
        try:
            df[col] = df[col].str.replace(',', '', regex=False)
            df[col] = df[col].astype('double')
        except AttributeError:
            continue

    return df

In [ ]:
def loading_v2(path: str, name: str) -> pd.DataFrame:
    renaming_dict = {
        'Datetime': 'date', 'Close': name, 'Volume': name + '_vol'
    }

    df = pd.read_csv(path, parse_dates=['Datetime']).drop(columns=['Adj Close'])

    df['Datetime'] = df.Datetime.dt.tz_localize(None) # remove timezone


    df = df.rename(columns=renaming_dict)
    df = df[renaming_dict.values()]
    
    df[name+'_chg'] = df[name].pct_change().fillna(0).multiply(100).round(2)
    df[name+'_vol_chg'] = df[name+'_vol'].pct_change().fillna(0).multiply(100).round(2)
    for col in df.columns[1:]:
        try:
            df[col] = df[col].str.replace(',', '', regex=False)
            df[col] = df[col].astype('double')
        except AttributeError:
            continue

    return df

In [ ]:
def loading_v3(path: str) -> pd.DataFrame:
    renaming_dict = {
        'Adj Close': 'Close'
    }

    df = pd.read_csv(path, parse_dates=['Date']).drop(columns=['Close'])
    df = df.rename(columns=renaming_dict)


    df['chg'] = df['Close'].pct_change().fillna(0).multiply(100).round(2)
    df['vol_chg'] = df['Volume'].pct_change().fillna(0).multiply(100).round(2)

    for col in df.columns[1:]:
        try:
            df[col] = df[col].str.replace(',', '', regex=False)
            df[col] = df[col].astype('double')
        except AttributeError:
            continue

    return df

In [ ]:
# # validate with
# # https://www.investing.com/equities/apple-computer-inc-historical-data?utm_source=google&utm_medium=cpc&utm_campaign=16816825626&utm_content=591978799158&utm_term=dsa-1546555491534_&GL_Ad_ID=591978799158&GL_Campaign_ID=16816825626&ISP=1&gad_source=1&gclid=CjwKCAiAxea5BhBeEiwAh4t5K3jLQNVzqbbCTLp-guh_jCrvv0cJI19-3Rh-kKb8spcB_GsFwx91ThoCGcIQAvD_BwE
# import os
# f = os.path.join('crypto_data', 'BTC-USD.csv')
# df = loading_v2(f, 'BTC-USD')

# df.tail(5)

,date,BTC-USD,BTC-USD_vol,BTC-USD_chg,BTC-USD_vol_chg
14307,2024-11-25 06:00:00,98210.679688,412405760,0.05,-53.23
14308,2024-11-25 07:00:00,98148.804688,500002816,-0.06,21.24
14309,2024-11-25 08:00:00,98337.335938,641888256,0.19,28.38
14310,2024-11-25 09:00:00,98575.945312,1784664064,0.24,178.03
14311,2024-11-25 10:00:00,98650.242188,261718016,0.08,-85.34
